In [19]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import numpy as np

CLUSTER_DISTANCE = 1000000
CAJAMAR = 3509205

def simple_map(ids, locations):
    folium_map = folium.Map(location=locations[ids[0]], zoom_start=4)
    for id in ids:
        folium.Marker(locations[id]).add_to(folium_map)
        
    return folium_map

def marker_color(is_present):
    if is_present:
        return 'green'
    else:
        return 'purple'

def clusterize_one(id, storages, dist, codes):
    if id in storages:
        return id
    storages_list = []
    for storage in storages:
        for idx,code in enumerate(codes):
            if code == storage:
                storages_list.append(idx)
                break
        
    candidate = -1
    for storage_idx in storages_list:
        if candidate == -1 or dist[id][candidate] > dist[id][storage_idx]:
            candidate = storage_idx
    return codes[candidate]
    
def clusterize_all(codes, storages, clients, dist):
    clustering = dict()
    for id in clients:
        center = clusterize_one(id, storages, dist, codes)
        if center != -1:
            clustering[id] = center
    return clustering
    
def clustered_map(cluster_ids, clients_ids, clustering, locations, is_present):
    folium_map = folium.Map(location=locations[clients_ids[0]], zoom_start=4)
    for id in clients_ids:
        folium.CircleMarker(locations[id],color=marker_color(is_present[id]),radius=1).add_to(folium_map)
    for id in cluster_ids:
        folium.CircleMarker(locations[id],color='red',radius=3).add_to(folium_map)
    folium.CircleMarker(locations[CAJAMAR],color='blue',radius=3).add_to(folium_map)
    for key in clustering:
        folium.PolyLine(locations=[locations[key], locations[clustering[key]]], color='blue', weight=0.2).add_to(folium_map)

    return folium_map
    
def read_input(codes):
    f = open("../data/input.in","r")
    n = int(f.readline())

    cod = np.zeros(n)
    pop = np.zeros(n)
    idh = [0.0] * n

    for i in range(n):
        item = f.readline().split()
        cod[i], pop[i], idh[i] = int(item[0]), int(item[1]), float(item[2])

    dist = [[] for _ in range(n)]
    for i in range(n):
        dist[i] = list(map(float, f.readline().split()))
    return dict(zip(codes,dist))
    
lats, lons, names, altitude = [],[],[],[]

df = pd.read_csv("../data/data.csv")

codes = df["Cód Cidades"].tolist()

lats = df["Latitude"].tolist()
lons = df["Longitude"].tolist()

is_present_list = [True if present == "sim" else False for present in df["Já é atendido pela Loggi?"].tolist()]
is_present = dict(zip(codes,is_present_list))

is_airport_list = [True if airport == "sim" else False for airport in df["Tem malha aerea?"].tolist()]
is_airport = dict(zip(codes,is_airport_list))

locations_list = [[lats[i],lons[i]] for i in range(len(lats))]
locations = dict(zip(codes, locations_list))
dist = read_input(codes)

output = open("../data/output3.out", "r")
n_storages = int(output.readline())
storages = list(map(int, output.readline().split()))
n_clients = int(output.readline())
clients = list(map(int, output.readline().split()))
    
#cluster_ids = [5002704]
#ids = [3550308, 3304557, 5300108, 2927408]   
#clustering = {5002704:3550308}
clustering = clusterize_all(codes, storages, clients, dist)
clustered_map(storages, clients, clustering, locations, is_present)



In [25]:
print(storages)

[1500800, 3300209, 3508504]
